In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time


body_types = ["Все типы кузовов", "Открытые", "Закрытые, без рефрижераторов", "Рефрижераторы"]
# body_types = ["Рефрижераторы"]


driver_path = './chromedriver.exe'  # Укажите путь к ChromeDriver
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

data_by_type = {}

try:
    for body_type in body_types:

        driver.get("...") # вместо точек ссылка на сайт

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".card"))
        )

        type_button = driver.find_element(By.CSS_SELECTOR, ".select-header")
        type_button.click()  

        WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, ".options-wrapper"))
        )

        options = driver.find_elements(By.CSS_SELECTOR, ".options-item")

        for option in options:
            if option.text.strip() == body_type:
                option.click()
                break

        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'За всё время')]")
        )).click()

        time.sleep(2)

        graph_area = driver.find_element(By.CSS_SELECTOR, ".responsive-graph")

        driver.execute_script("arguments[0].scrollIntoView();", graph_area)

        window_width = driver.get_window_size()['width']

        data = []

        action = ActionChains(driver)

        for x_position in range(0, window_width, 1): 
            try:
                action.move_to_element_with_offset(graph_area, x_position - graph_area.location['x'] - 100, 0).perform()

                time.sleep(0.5)

                tooltip = driver.find_elements(By.CSS_SELECTOR, ".recharts-tooltip-wrapper")
                if tooltip:
                    index = tooltip[0].find_element(By.CSS_SELECTOR, ".graph-tooltip-index").text
                    date = tooltip[0].find_element(By.CSS_SELECTOR, ".graph-tooltip-date").text
                    data.append({"Index": index, "Date": date})
            except Exception as e:
                pass

        data_by_type[body_type] = pd.DataFrame(data).drop_duplicates()

        time.sleep(2)

except Exception as e:
    pass

finally:
    driver.quit()

with pd.ExcelWriter("ATISU_all.xlsx") as writer:
    for body_type, df in data_by_type.items():
        if not df.empty:  
            sheet_name = body_type[:30]  
            df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Данные сохранены в файл ATISU_all.xlsx")